# Optimisation des warehouses

## Import des modules et des données

In [1]:
from sklearn.cluster import KMeans
import folium
import pandas as pd
import edc
from edc import maps
import branca.colormap as cm
%load_ext autoreload
%autoreload 2

In [26]:
df_orders_lines = pd.read_csv('..\data\intermediate_data\df_orders_lines.csv')
df_carte_point = pd.read_csv('..\data\intermediate_data\df_carte_point.csv')
df_cities = pd.read_csv("../data/cities.csv")

## Calcul des nouveaux warehouses

On utilise un kmeans pour segmenter le territoire en 5 en fonction des distances entre les villes et les délais de livraison. On récupère les centroïdes de ces clusters.

In [31]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_orders_lines[['lat_delivery','lng_delivery']],df_orders_lines.order_total_volume)
new_warehouse = pd.DataFrame(kmeans.cluster_centers_,columns=['lat','lng'])

In [32]:
df_orders_lines['new_warehouse'] = kmeans.labels_
df_orders_lines.rename(columns={'lat_delivery':'lat' , 'lng_delivery':'lng'},inplace=True)
new_warehouse['index_warehouse'] = new_warehouse.index

En noir les entrepôts actuels, en couleur les entrepôts optimaux.

In [53]:
m = folium.Map()

colormap = cm.LinearColormap(colors=['white','black'], vmin=df_carte_point.order_total_volume.min(),vmax=df_carte_point.order_total_volume.max())
colormap.caption = 'order total volume'

df_orders_lines.apply(lambda point : maps.plotDot(m,point,color=maps.colors[point.new_warehouse]), axis = 1)
m.add_child(colormap,name='order total volume')

#Set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

df_cities[df_cities.is_warehouse].apply(lambda point : maps.plotMarker(m,point,color='black'), axis = 1)
new_warehouse.apply(lambda point : maps.plotMarker(m,point,color = maps.colors[int(point.index_warehouse)],with_tooltip=None), axis = 1)

m